In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier #support from multiclass
import time
from sklearn.svm import SVC


class DataLoader(object):
    def __init__(self, path_main, ticker):
        self.main_path = path_main
        self.ticker = ticker

        self.features_labels_path = os.path.join(self.main_path, 'features_models')
        self.features_path = os.path.join(self.features_labels_path, 'features')
        # collection of per symbol non directional labels
        self.labels_path = os.path.join(self.features_labels_path, 'labels', self.ticker, 'NON_DIRECTIONAL')
        self.symbol_features_path = os.path.join(self.features_labels_path, 'features', self.ticker, 'MODEL_BASED')
        # list of all the model -oos hmm feature dates - each folder is a collection of oos feature dates
        self.oos_dates_list = os.listdir(self.symbol_features_path)

    def ticker_features(self, model_date, date):
        # need to make this a lot more flexible with number of states
        if model_date < date:
            file_name = "_".join((self.ticker, '3', 'states', 'features', 'date:', date, 'now:', '20181224', '.pickle'))
            file_loc = os.path.join(self.symbol_features_path, str(model_date), file_name)
            with open(file_loc, 'rb') as handle:
                ticker_features = pickle.load(handle)
        else:
            print('Loading Feature Date which is in-sample. Change your Model Date')
        return ticker_features

    def ticker_labels_csv(self, date):
        file_loc = os.path.join(self.labels_path, str(date) + '.csv')
        ticker_labels = pd.read_csv(file_loc, index_col=0)
        return ticker_labels

    @staticmethod
    def open_pickle_file(path, pickle_file):
        file_loc = os.path.join(path, pickle_file)
        pickle_to_file = pickle.load(open(file_loc, "rb"))
        return pickle_to_file

    @staticmethod
    def get_date_from_file(file_, numb_):
        return os.path.splitext(file_[numb_])[0]


class MarketFeatures(object):
    # a class to be expanded that uses features for base case -market based only-indicators/features
    """"Requires:
    a dataframe that has TradedPrice And Volume columns
    symbol - A stock symbol on which to form a strategy on.
    short_window - Lookback period for short moving average.
    long_window - Lookback period for long moving average.
    """

    def __init__(self, df):
        #         self.ticker = ticker
        self.df = df

    def load_data(self):
        pass

    def ma_spread(self, short_window=5, long_window=20):
        # function that produces the MA spread, which can be used on its own or as an input for MACD
        short_rolling_px = self.df['TradedPrice'].rolling(window=short_window).mean()
        long_rolling_px = self.df['TradedPrice'].rolling(window=long_window).mean()
        px_name = "_".join(('px_indx', str(short_window), str(long_window)))
        self.df[px_name] = long_rolling_px - short_rolling_px
        return self.df

    def obv_calc(self):
        # on balance volume indicator
        self.df['SignedVolume'] = self.df['Volume'] * np.sign(self.df['TradedPrice'].diff()).cumsum()
        self.df['SignedVolume'].iat[1] = 0
        self.df['OBV'] = self.df['SignedVolume']  # .cumsum()
        self.df = df.drop(columns=['SignedVolume'])
        return self.df

    def chaikin_mf(self, period=5):
        # Chaikin money flow indicator
        self.df["MF Multiplier"] = (self.df['TradedPrice'] - (self.df['TradedPrice'].expanding(period).min()) \
                                    - (self.df['TradedPrice'].expanding(period).max() \
                                       - self.df['TradedPrice'])) / (
                                           self.df['TradedPrice'].expanding(period).max() - self.df[ \
                                            'TradedPrice'].expanding(period).min())
        self.df["MF Volume"] = self.df['MF Multiplier'] * df['Volume']
        self.df['CMF_' + str(period)] = self.df['MF Volume'].sum() / self.df["Volume"].rolling(period).sum()
        self.df = self.df.drop(columns=['MF Multiplier', 'MF Volume'])
        return self.df


class FitModels(object):

    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    # # Train a SVM classification model

    def best_kernel_ridge(self, kernel_choice):
        kr_clf = OneVsRestClassifier(GridSearchCV(KernelRidge(kernel=str(kernel_choice)), cv=10,
                                                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                                                              "gamma": np.logspace(-2, 2, 5)})).fit(self.X_train,
                                                                                                    self.y_train)

        return kr_clf

    def best_svm_clf(self, kernel_choice):
        param_grid = dict(kernel=[str(kernel_choice)],
                          C=[1, 5, 10, 25, 50, 100],
                          gamma=[0.0001, 0.001, 0.01, 0.02, 0.05, 0.01])

        clf = OneVsRestClassifier(
            GridSearchCV(SVC(class_weight='balanced'), param_grid, verbose=1, n_jobs=-1, cv=10)) \
            .fit(self.X_train, self.y_train)
        return clf

    def best_gradient_boost_clf(self):
        # this needs to be written properly- just a baseline placeholder here!
        GBR = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                        max_depth=4, max_features='sqrt',
                                        min_samples_leaf=15, min_samples_split=10, loss='huber',
                                        random_state=5)

        gb_boost_clf = OneVsRestClassifier(GBR).fit(self.X_train, self.y_train)

        return gb_boost_clf

    def best_MKL_clf(self):
        pass

    def best_knn_clf(self):
        pass

    def best_random_forest_clf(self):
        pass

    def run_cv(self, clf_class, **kwargs):
        # Construct a kfolds object
        kf = KFold(len(self.y_train), n_folds=10, shuffle=True)
        y_pred = self.y_train.copy()

        # Iterate through folds
        for train_index, test_index in kf:
            X_train_local, X_test_local = self.X_train[train_index], self.X_train[test_index]
            y_train_local = self.y_train[train_index]
            # Initialize a classifier with key word arguments
            clf = clf_class(**kwargs)
            clf.fit(self.X_train, self.y_train)
            y_pred[test_index] = clf.predict(X_test_local)
        return y_pred


class PredictModels(FitModels):
    def __init__(self):
        pass


def no_nans(label):
    return np.sum(np.isnan(label))


def remove_last_element(arr):
    return arr[np.arange(arr.size - 1)]

sc = StandardScaler()



In [9]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file
def common_member(a, b): 
      
    a_set = set(a) 
    b_set = set(b) 
      
    # check length  
    if len(a_set.intersection(b_set)) > 0: 
        return(a_set.intersection(b_set))   
    else: 
        return("no common elements") 

In [83]:

if __name__ == '__main__':
    symbol = ftse_symbols[1]

    # sort out locations
    # data_dir: main directory , data_only_drive: the big drive where everything is saved

    data_dir = os.getenv('FINANCE_DATA')
    data_only_drive = '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2' #external date only drive

    # from the main directory select all the symbols that are finishing in .L for FTSE
    ftse_symbols = [s for s in os.listdir(data_dir) if s.endswith('.L')]

    # this is the central location for all the features/models/predictions
    features_models = os.path.join(data_dir, 'features_models')

    # this is the central location for all the labels
    labels = os.path.join(features_models, 'labels')
    # this is the central location for all the features
    features = os.path.join(features_models, 'features')
    # this is the central location for all the saved HMM models
    hmm_models = os.path.join(features_models, 'models', 'HMM')
    # location to save results
    model_save_loc = os.path.join(data_only_drive, 'Data','features_models','models')

    # Picked one random symbol - this has to be done over all symbols:

    symbol_features_path = os.path.join(features, ftse_symbols[1], 'MODEL_BASED')
    print(symbol_features_path) #remove

    datacls = DataLoader(path_main=data_dir, ticker=ftse_symbols[1]) #test symbol -create class

    # list of dates that hmm models have been fitted and used out of sample
    model_dates = datacls.oos_dates_list
    #  picking one such hmm model i.e picking an OOS date

    for model_date_idx, model_date in enumerate(model_dates):
        print(model_date)
        t0 = time.time()
        #  path for hmm model features - so concatenates, the specific symbol->features->HMM model(date)->OOS dates
        feature_files_path = os.path.join(symbol_features_path, model_dates[model_date_idx])

        # get all the feature files for each model date
        list_of_feature_files_for_model_date = os.listdir(os.path.join(symbol_features_path, model_dates[model_date_idx]))
        # # get the dates for those feature files- so this returns a list of dates in the format 'YYYY MM DD'
        feature_dt_model_date = [list_of_feature_files_for_model_date[model_date_idx].split("_")[5] for idx, _ in
                                 enumerate(list_of_feature_files_for_model_date)]
        # # pick a feature file
        # print(feature_dt_model_date)  # remove
#         for date_idx, date_features in enumerate(feature_dt_model_date):
#             test_pickle_file = list_of_feature_files_for_model_date[date_idx]
#             print('test file:', test_pickle_file)
#         # # get all the features in one go

       # features_tuple = datacls.open_pickle_file(path=feature_files_path, pickle_file=test_pickle_file)
    # # get labels file
    # df = datacls.ticker_labels_csv(date=list_of_feature_files_for_model_date[1].split("_")[5])
    # df_w_market_features = MarketFeatures(
    #     df=MarketFeatures(df=MarketFeatures(df=df).obv_calc()).chaikin_mf()).ma_spread()
    #
    # features_df = pd.concat([features_tuple[0], features_tuple[1], features_tuple[2], \
    #                          features_tuple[3]], axis=1, sort=False)
    #
    # df_concat = pd.concat([features_df, df_w_market_features], axis=1, sort='False').dropna()
    #
    # # right order of things
    #
    # y = np.asanyarray(df_concat['Duration'])
    # df_final = df_concat.drop(columns=['TradedPrice', 'Duration', 'TradedTime'])
    # X = df_final
    #
    # from sklearn.svm import SVR
    #
    # np.random.seed(0)
    # svr = GridSearchCV(SVR(kernel='rbf'), cv=5,
    #                    param_grid={"C": [1e0, 1e1, 1e2, 1e3],
    #                                "gamma": np.logspace(-2, 2, 5)})
    # svr.fit(X , y)
    #


/media/ak/WorkDrive/Data/features_models/features/CEY.L/MODEL_BASED
20171006
20170922
20180405
20180227
20170711
20170216
20171020
20170904
20180418


IndexError: list index out of range

In [13]:
symbol_features_path

'/media/ak/WorkDrive/Data/features_models/features/CEY.L/MODEL_BASED'

In [ ]:
for model_date_idx, model_date in enumerate(model_dates):
    compute_date= '20181224'
    print(model_date)
    t0 = time.time()
    #  path for hmm model features - so concatenates, the specific symbol->features->HMM model(date)->OOS dates
    feature_files_path = os.path.join(symbol_features_path, model_dates[model_date_idx])
    print(feature_files_path)
            # get all the feature files for each model date
    list_of_feature_files_for_model_date = os.listdir(os.path.join(symbol_features_path, model_dates[model_date_idx]))
    labels_dates = [os.listdir(datacls.labels_path)[idx].split(".")[0] \
                    for idx,_ in enumerate(os.listdir(datacls.labels_path))]

        # # get the dates for those feature files- so this returns a list of dates in the format 'YYYY MM DD'
    feature_dts = [list_of_feature_files_for_model_date[idx].split("_")[5] for idx, _ in
                                 enumerate(list_of_feature_files_for_model_date)]
    common_dates = common_member(labels_dates, feature_dts) #this is pretty critical. we can only use common dates

    for date_idx, common_date in enumerate(common_dates):
        features_pickle_file = "_".join((symbol,'3_states_features_date:',common_date,'now:',compute_date,'.pickle'))
        print('this is the right date:', common_date)
        print('test file:', features_pickle_file)
        test_labels_file = os.path.join(datacls.labels_path, common_date+'.csv')
        exists = os.path.isfile(test_labels_file)
        if exists:
            print('test labels file:', test_labels_file)
        # Store configuration file values
        else:
            print('problem in:', common_date)
                     

        features_tuple = datacls.open_pickle_file(path=feature_files_path, pickle_file=features_pickle_file)
        # # get labels file
        df = datacls.ticker_labels_csv(date=common_date)
        df_w_market_features = MarketFeatures( \
               df=MarketFeatures(df=MarketFeatures(df=df).obv_calc()).chaikin_mf()).ma_spread()

        features_df = pd.concat([features_tuple[0], features_tuple[1], features_tuple[2], features_tuple[3]], axis=1, sort=False)
        # concatenate features from model and market features

        df_concat = pd.concat([features_df, df_w_market_features], axis=1, sort='False').dropna()
        print('all done, go fit your model:')

print time.time()-t0

20171006
/media/ak/WorkDrive/Data/features_models/features/CEY.L/MODEL_BASED/20171006
('this is the right date:', '20180418')
('test file:', 'CEY.L_3_states_features_date:_20180418_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180418.csv')
all done, go fit your model:
('this is the right date:', '20180419')
('test file:', 'CEY.L_3_states_features_date:_20180419_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180419.csv')
all done, go fit your model:
('this is the right date:', '20180412')
('test file:', 'CEY.L_3_states_features_date:_20180412_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180412.csv')
all done, go fit your model:
('this is the right date:', '20180413')
('test file:', 'CEY.L_3_states_features_date:_20180413_now:_20181224_.pickle')
('test labels file:', '/med

all done, go fit your model:
('this is the right date:', '20171031')
('test file:', 'CEY.L_3_states_features_date:_20171031_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171031.csv')
all done, go fit your model:
('this is the right date:', '20180202')
('test file:', 'CEY.L_3_states_features_date:_20180202_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180202.csv')
all done, go fit your model:
('this is the right date:', '20180201')
('test file:', 'CEY.L_3_states_features_date:_20180201_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180201.csv')
all done, go fit your model:
('this is the right date:', '20180207')
('test file:', 'CEY.L_3_states_features_date:_20180207_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20171018')
('test file:', 'CEY.L_3_states_features_date:_20171018_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171018.csv')
all done, go fit your model:
('this is the right date:', '20171019')
('test file:', 'CEY.L_3_states_features_date:_20171019_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171019.csv')
all done, go fit your model:
('this is the right date:', '20171016')
('test file:', 'CEY.L_3_states_features_date:_20171016_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171016.csv')
all done, go fit your model:
('this is the right date:', '20171017')
('test file:', 'CEY.L_3_states_features_date:_20171017_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20171023')
('test file:', 'CEY.L_3_states_features_date:_20171023_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171023.csv')
all done, go fit your model:
('this is the right date:', '20171020')
('test file:', 'CEY.L_3_states_features_date:_20171020_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171020.csv')
all done, go fit your model:
('this is the right date:', '20171027')
('test file:', 'CEY.L_3_states_features_date:_20171027_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171027.csv')
all done, go fit your model:
('this is the right date:', '20171026')
('test file:', 'CEY.L_3_states_features_date:_20171026_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20171017')
('test file:', 'CEY.L_3_states_features_date:_20171017_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171017.csv')
all done, go fit your model:
('this is the right date:', '20170831')
('test file:', 'CEY.L_3_states_features_date:_20170831_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170831.csv')
all done, go fit your model:
('this is the right date:', '20170830')
('test file:', 'CEY.L_3_states_features_date:_20170830_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170830.csv')
all done, go fit your model:
('this is the right date:', '20171012')
('test file:', 'CEY.L_3_states_features_date:_20171012_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

...

all done, go fit your model:
('this is the right date:', '20180214')
('test file:', 'CEY.L_3_states_features_date:_20180214_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180214.csv')
all done, go fit your model:
('this is the right date:', '20180215')
('test file:', 'CEY.L_3_states_features_date:_20180215_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180215.csv')
all done, go fit your model:
('this is the right date:', '20180216')
('test file:', 'CEY.L_3_states_features_date:_20180216_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180216.csv')
all done, go fit your model:
('this is the right date:', '20180209')
('test file:', 'CEY.L_3_states_features_date:_20180209_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20170921')
('test file:', 'CEY.L_3_states_features_date:_20170921_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170921.csv')
all done, go fit your model:
('this is the right date:', '20170920')
('test file:', 'CEY.L_3_states_features_date:_20170920_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170920.csv')
all done, go fit your model:
('this is the right date:', '20171013')
('test file:', 'CEY.L_3_states_features_date:_20171013_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20171013.csv')
all done, go fit your model:
('this is the right date:', '20180420')
('test file:', 'CEY.L_3_states_features_date:_20180420_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20170927')
('test file:', 'CEY.L_3_states_features_date:_20170927_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170927.csv')
all done, go fit your model:
('this is the right date:', '20170728')
('test file:', 'CEY.L_3_states_features_date:_20170728_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170728.csv')
all done, go fit your model:
('this is the right date:', '20170921')
('test file:', 'CEY.L_3_states_features_date:_20170921_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170921.csv')
all done, go fit your model:
('this is the right date:', '20170920')
('test file:', 'CEY.L_3_states_features_date:_20170920_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20170815')
('test file:', 'CEY.L_3_states_features_date:_20170815_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170815.csv')
all done, go fit your model:
('this is the right date:', '20170814')
('test file:', 'CEY.L_3_states_features_date:_20170814_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170814.csv')
all done, go fit your model:
('this is the right date:', '20170817')
('test file:', 'CEY.L_3_states_features_date:_20170817_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20170817.csv')
all done, go fit your model:
('this is the right date:', '20170816')
('test file:', 'CEY.L_3_states_features_date:_20170816_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

all done, go fit your model:
('this is the right date:', '20180413')
('test file:', 'CEY.L_3_states_features_date:_20180413_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180413.csv')
all done, go fit your model:
('this is the right date:', '20180410')
('test file:', 'CEY.L_3_states_features_date:_20180410_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180410.csv')
all done, go fit your model:
('this is the right date:', '20180411')
('test file:', 'CEY.L_3_states_features_date:_20180411_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIRECTIONAL/20180411.csv')
all done, go fit your model:
('this is the right date:', '20180416')
('test file:', 'CEY.L_3_states_features_date:_20180416_now:_20181224_.pickle')
('test labels file:', '/media/ak/WorkDrive/Data/features_models/labels/CEY.L/NON_DIR

In [18]:
list_CEY= os.listdir(symbol_features_path)

In [40]:
labels = os.listdir(datacls.labels_path)


In [38]:
common_member(list_CEY, labels)

'no common elements'

In [47]:
labels_test=os.listdir(datacls.labels_path)

In [46]:
features_test =os.listdir(os.path.join(symbol_features_path,os.listdir(symbol_features_path)[1]))

In [49]:
len(labels_test)

140

In [63]:
features_dates = [features_test[idx].split("_")[5]  for idx,_ in enumerate(features_test)]
labels_dates = [labels_test[idx].split(".")[0]  for idx,_ in enumerate(labels_test)]

In [66]:
len(common_member(features_dates, labels_dates)) #this is what I need- to essentially read the common dates and go through those!


61

In [78]:
labels_dates = [os.listdir(datacls.labels_path)[idx].split(".")[0]  for idx,_ in enumerate(os.listdir(datacls.labels_path))]

In [79]:
labels_dates

['20170928',
 '20171027',
 '20170803',
 '20170711',
 '20170824',
 '20170808',
 '20170210',
 '20171004',
 '20170705',
 '20171024',
 '20180220',
 '20170904',
 '20170224',
 '20170915',
 '20170913',
 '20171020',
 '20170802',
 '20171009',
 '20170830',
 '20170223',
 '20171003',
 '20170810',
 '20170727',
 '20170213',
 '20170712',
 '20170831',
 '20170829',
 '20180411',
 '20170801',
 '20170704',
 '20170901',
 '20171018',
 '20170821',
 '20180201',
 '20170908',
 '20170228',
 '20170914',
 '20180221',
 '20170725',
 '20170728',
 '20170227',
 '20170222',
 '20171026',
 '20180205',
 '20170922',
 '20170209',
 '20180419',
 '20180405',
 '20170703',
 '20170825',
 '20170207',
 '20180212',
 '20170719',
 '20171010',
 '20170221',
 '20170816',
 '20180417',
 '20180219',
 '20170815',
 '20180413',
 '20170907',
 '20170706',
 '20170710',
 '20171031',
 '20170217',
 '20171016',
 '20180216',
 '20180416',
 '20170811',
 '20170921',
 '20180420',
 '20170203',
 '20171013',
 '20180227',
 '20180213',
 '20180206',
 '20170720',